In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [48]:
df_train = pd.read_csv('CompetenciaECI/Entrenamieto_ECI_2020.csv', index_col = 'Opportunity_ID')
df_val = pd.read_csv('CompetenciaECI/Validacion_ECI_2020.csv', index_col = 'Opportunity_ID')
target = 'Stage'
predictors = [x for x in df_train.columns if x not in [target]]
y = df_train.loc[:,target].copy()
y = (y=='Closed Won') * 1 
df_train = df_train.loc[:, predictors]
df = pd.concat([df_train,df_val], axis=0)
print('Tamaños de entrenamiento')
print(f'Tamanio de X: {df_train.shape}')
print(f'Tamanio de X_val: {df_val.shape}')
print(f'Tamanio de y: {y.shape}')

Tamaños de entrenamiento
Tamanio de X: (16947, 50)
Tamanio de X_val: (2551, 50)
Tamanio de y: (16947,)


## Valores faltantes

In [10]:
def nan_values(df):
    total = df.isnull().sum().sort_values(ascending=False)
    percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
    missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    print(missing_data.head(6))
nan_values(df_train)
nan_values(df_val)
nan_values(df)

                 Total   Percent
ASP               3209  0.189355
ASP_(converted)   3209  0.189355
Total_Amount        59  0.003481
Prod_Category_A      0  0.000000
Delivery_Terms       0  0.000000
Quote_Type           0  0.000000
                          Total   Percent
ASP                          84  0.032928
ASP_(converted)              84  0.032928
Prod_Category_A               0  0.000000
Opportunity_Created_Date      0  0.000000
Delivery_Terms                0  0.000000
Quote_Type                    0  0.000000
                 Total   Percent
ASP               3293  0.168889
ASP_(converted)   3293  0.168889
Total_Amount        59  0.003026
Prod_Category_A      0  0.000000
Delivery_Terms       0  0.000000
Quote_Type           0  0.000000


## Analisis de variables numericas 

## Valores None en variables numericas

In [11]:
numerical_cols = ['Sales_Contract_No', 'ASP', 'ASP_(converted)', 
              'TRF', 'Total_Amount', 'Total_Taxable_Amount', 'ID']

In [15]:
def none_values(df):
    df_num = df[numerical_cols]
    total = df_num.apply(lambda x: x=='None').sum().sort_values(ascending=False)
    percent = (total/df_num.apply(lambda x: x=='None').count()).sort_values(ascending=False)
    missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    print(missing_data.head(12))
none_values(df_train)
none_values(df_val)
none_values(df)

                      Total   Percent
Sales_Contract_No      6973  0.411459
ID                        0  0.000000
Total_Taxable_Amount      0  0.000000
Total_Amount              0  0.000000
TRF                       0  0.000000
ASP_(converted)           0  0.000000
ASP                       0  0.000000
                      Total  Percent
Sales_Contract_No      1213   0.4755
ID                        0   0.0000
Total_Taxable_Amount      0   0.0000
Total_Amount              0   0.0000
TRF                       0   0.0000
ASP_(converted)           0   0.0000
ASP                       0   0.0000
                      Total   Percent
Sales_Contract_No      8186  0.419838
ID                        0  0.000000
Total_Taxable_Amount      0  0.000000
Total_Amount              0  0.000000
TRF                       0  0.000000
ASP_(converted)           0  0.000000
ASP                       0  0.000000


/home/martin/anaconda3/envs/tesis_env/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/home/martin/anaconda3/envs/tesis_env/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/home/martin/anaconda3/envs/tesis_env/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


## Analisis de variables categoricas 

In [45]:
binary_cols = ['Pricing, Delivery_Terms_Quote_Appr', 'Pricing, Delivery_Terms_Approved', 
           'Bureaucratic_Code_0_Approval','Bureaucratic_Code_0_Approved']

categorical_cols = ['Region','Territory','Bureaucratic_Code', 
                'Source ','Billing_Country','Account_Name',
                'Opportunity_Name', 'Account_Owner',
               'Opportunity_Owner','Account_Type', 'Opportunity_Type', 
                'Quote_Type', 'Delivery_Terms', 'Brand', 'Product_Type', 
                'Size','Product_Category_B', 'Currency', 'Last_Modified_By',
                'Product_Family', 'Product_Name', 'ASP_Currency',
                'ASP_(converted)_Currency','Delivery_Quarter',
                'Total_Taxable_Amount_Currency',
                'Prod_Category_A', 'Total_Amount_Currency'
               ]
cols = binary_cols + categorical_cols

In [46]:
for d in [df, df_train, df_val]:
    d[cols] = d[cols].astype('category')

### Como cambian las variables categoricas dentro de una misma oportunidad

In [47]:
# Que categorias tienen presentes diferentes valores en una misma oportunidad?
# Product_Family, Product_Name y Delivery_Quarter

d = df.select_dtypes(include=['category'])
group = d.groupby(d.index)
for col in d.columns:
    print(col)
    print(group[col].apply(lambda x: len(set(x))!=1).sum())

Region
0
Territory
0
Pricing, Delivery_Terms_Quote_Appr
0
Pricing, Delivery_Terms_Approved
0
Bureaucratic_Code_0_Approval
0
Bureaucratic_Code_0_Approved
0
Bureaucratic_Code
0
Source 
0
Billing_Country
0
Account_Name
0
Opportunity_Name
0
Sales_Contract_No
0
Account_Owner
0
Opportunity_Owner
0
Account_Type
0
Opportunity_Type
0
Quote_Type
0
Delivery_Terms
0
Brand
0
Product_Type
0
Size
0
Product_Category_B
0
Currency
0
Last_Modified_By
0
Product_Family
2639
Product_Name
2811
ASP_Currency
0
ASP_(converted)_Currency
0
Delivery_Quarter
421
Total_Amount_Currency
0
Total_Taxable_Amount_Currency
0
Prod_Category_A
0


## Valores None en columnas categoricas/binarias

In [ ]:
#df[numerical_cols] = df[numerical_cols].astype('float64')
def none_values(df):
    df_cat = df[cols]
    total = df_cat.apply(lambda x: x=='None').sum().sort_values(ascending=False)
    percent = (total/df_cat.apply(lambda x: x=='None').count()).sort_values(ascending=False)
    missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    print(missing_data.head(12))
none_values(df_train)
none_values(df_val)
none_values(df)

## Relacion entre las categorias y la variable objetivo

In [ ]:
d = pd.concat([df_train, y], axis=1)
for col in d.select_dtypes(include='category').columns:
    groups = d.groupby(col)
    print(d.groupby(col).agg({col:'count', 'Stage':'mean'}).sort_values('Stage', ascending=False).head(30))


### Cantidad de categorias por variable categorica

In [ ]:
df.select_dtypes(include='category').nunique().sort_values(ascending=False)

### Relation Region/Territorio
- Cuanto el Territorio es None la region en gral es Japan
- Japon tiene su propio territorio
- Hay 5000 Nones en Territorio, siendo la mayoria de la region Japan
- Decision: se completan los Nones con el nombre de las Region

In [ ]:
# territorios por region
df.loc[:,['Territory','Region']].groupby('Region')['Territory'].apply(set)

In [ ]:
# a que region corresponden los valores nulos de los territorios?
df.loc[df['Territory']=='None',['Territory','Region']].groupby('Region')['Region'].count()

In [ ]:
# cuando la region es japon, hay muchos nones 
df.loc[df['Region']=='Japan',['Territory','Region']].groupby('Territory')['Territory'].count().sort_values(ascending=False)

## Cantidad de registros por tipo en cada categoria y cantidad de categorias

In [ ]:
def print_categorical_values(cat):
    groups = df.groupby([cat])[cat]
    print(groups.size().sort_values(ascending=False))
    print(len(groups))
    print()

In [ ]:
for cat in categorical_cols:
    print_categorical_values(cat)

## Analisis variables fechas

In [ ]:
datetime_cols = ['Account_Created_Date', 'Opportunity_Created_Date',
             'Last_Activity','Quote_Expiry_Date', 'Last_Modified_Date',
             'Planned_Delivery_Start_Date', 'Planned_Delivery_End_Date',
             'Actual_Delivery_Date']

In [ ]:
for d in [df,df_val,df_train]:
    for col in datetime_cols:
        d[col] = pd.to_datetime(d[col])


In [ ]:
def none_values(df):
    f = lambda x: x.isnull()
    df_date = df[datetime_cols]
    total = df_date.apply(f).sum().sort_values(ascending=False)
    percent = (total/df_date.apply(f).count()).sort_values(ascending=False)
    missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    print(missing_data)
none_values(df_train)
none_values(df_val)
none_values(df)

In [ ]:
# Quiero ver cual es la relacion entre Quote_Expiry_Date y la variable objectivo
# en 12mil de los casos de entrenamiento, cuando la fecha de vencimiento del presupuesto esta seteada, la oportunidad se gana
quote_not_set = df_train['Quote_Expiry_Date'].isnull()
print((quote_not_set != y.astype('bool')).sum())

# y siempre se mantiene igual para la misma oportunidad
print(quote_not_set.groupby(quote_not_set).apply(lambda x: len(set(x))!=1).sum())

In [ ]:
# Las unicas variables de fecha que varian para una misma son Planned_Delivery_Start_Date y Planned_Delivery_End_Date
d = pd.concat([df_train, df_val], axis=0).select_dtypes(include=['datetime'])
for col in d.columns:
    print(col)
    print(d.groupby(d.index)[col].apply(lambda x: len(set(x))>1).sum())

## Posibles target leackage
- Submitted_for_Approval: casi toda 0
- Delivery_Terms: nop
- Quote_Expiry_Date
- Actual_Delivery_Date: es puro NaT
- Total_Power: no esta en el dataset (tampoco la categorica correspondiente)
- Sales_Contract_No

In [ ]:
# los tipos de terminos no muestran un especial inclinacion haciia las oportunidades perdidas o ganadas
d = pd.concat([df_train, y], axis=1)
d.groupby('Delivery_Terms').agg({'Delivery_Terms':'count', 'Stage':'mean'}).sort_values('Stage', ascending=False)

In [51]:
# Esta parece ser el target Leackage, los productos que 
# no tienen Sales_Contract_No tienen un bajisimo numero de exito
d = pd.concat([df_train, y], axis=1).loc[:,['Sales_Contract_No','Stage']]
d['isNone'] = d.Sales_Contract_No == 'None'
d = d.groupby('isNone').agg(['count','mean'])
d.sort_values(d.columns[0], ascending=False)
# sigo en la proxima celda

Stage          
       count      mean
isNone                
False   9974  0.941949
True    6973  0.019791

In [49]:
d = pd.concat([df_train, df_val], axis=0)
d.Sales_Contract_No == 'None'

Opportunity_ID
0         True
1        False
2        False
3         True
4         True
         ...  
12364    False
12365    False
12366    False
12367     True
12368     True
Name: Sales_Contract_No, Length: 19498, dtype: bool

# Variable objetivo

In [ ]:
print('Posibles valores de la variable objetivo')
print(y.groupby(y).count())

In [ ]:
#prepare y
y.loc[y == 'Closed Won'] = 1
y.loc[y != 'Closed Won'] = 0
y.groupby(y).count()